In [13]:
import skimage
import os
import pandas as pd
import numpy as np
from IPython.display import display
import matplotlib
import matplotlib.pyplot as plt
from skimage import data
from skimage import io
import glob
from skimage.viewer import ImageViewer
import cv2
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
%matplotlib inline

In [2]:
#Define List of Users and define File-Path to Users Folder
list_users = ['ana_brandine', 'vicky_regouli', 'luismiguelpss', 'ilariabiagini', 
              'emnegg', 'kerendhahn', 'agiorgina', 'roulamatta', 'jussbieber9827', 
              'eremiaheidr', 'eunhuiheo', 'anastasiakaps', 'achaelilsone', 'orit_talbi',
              'sorayaalassmi', 'altonolnlis', 'vaso1977', 'theunrealobserver', 'nsb.koc',
              'vivpeng', 'amrynevillek', 'danalev7', 'irienyree', 'lilachturgeman', 
              'emel_karakoc', 'thiswhomustbekept', 'j_f_lil', 'ulietteearneye', 
              'gilanaz', 'sarrahdolly', 'alexchahine97', 'photographerarson', 
              'angecanindo', 'fiona_smithson', 'chelsea_xu620']

path = "C:/Users/karan/Desktop/Dropbox/Harvard/Capstone_EmpSirenuse/Datasets/sample_users_media/"
dest_path = r"C:/Users/karan/Desktop/Dropbox/Harvard/Capstone_EmpSirenuse/Clustering_"

In [5]:
#Helper Function to get images from path
def get_images(path, list_users):
    #Get all Images of Users in the List
    user_imgs = {}
    for i in list_users:
        temp_path = path + i
        for j in glob.glob(temp_path + '/*.jpg'):
            file_name = j.replace(temp_path,'')[1:]
            img = io.imread(j)
            user_imgs[(i,file_name)] = img
    return user_imgs 


#Get Images from User List and Path
user_imgs = get_images(path, list_users)
print("Number of images loaded:", len(user_imgs))

Number of images loaded: 4204


In [6]:
#Helper function to convert image to d-dimension vector
def convert(user_imgs):
    user_features = []
    for i in user_imgs.items():
        r_mean, r_std, r_med = np.mean(i[1][:,:,0].ravel()), np.std(i[1][:,:,0].ravel()), np.median(i[1][:,:,0].ravel())
        g_mean, g_std, g_med  = np.mean(i[1][:,:,1].ravel()), np.std(i[1][:,:,1].ravel()), np.median(i[1][:,:,1].ravel())
        b_mean, b_std, b_med  = np.mean(i[1][:,:,2].ravel()), np.std(i[1][:,:,2].ravel()), np.median(i[1][:,:,2].ravel())
        canny = np.mean(np.ravel(cv2.Canny(cv2.cvtColor(i[1], cv2.COLOR_BGR2HSV),100,200,L2gradient = True)))
        user_features.append(np.array([i[0][0],i[0][1], r_mean, r_std, r_med, g_mean, g_std, g_med, b_mean, b_std, b_med, canny]))
    return user_features
    
#Convert Images
user_features = convert(user_imgs)
print("Feature Vectors Created.")

Feature Vectors Created.


In [7]:
#Create Dataframe of Features
df = pd.DataFrame(user_features)
df.columns = ["User_Handle","URL","R_Mean", "R_STD", "R_MED", "G_Mean", "G_STD", "G_MED", "B_Mean", "B_STD", "B_MED", "Canny"]
display(df.head())

,User_Handle,URL,R_Mean,R_STD,R_MED,G_Mean,G_STD,G_MED,B_Mean,B_STD,B_MED,Canny
0,ana_brandine,25037786_149039252556440_4293334594247196672_n...,132.561698345,57.6752993548,129.0,114.603278557,56.8672587811,101.0,104.813399871,55.5512984082,85.0,7.39290025088
1,ana_brandine,25038631_157219874933240_4514763492913315840_n...,23.5705837674,54.3037901125,5.0,101.522250705,53.6189443253,101.0,170.55649821,51.737780211,181.0,22.7942911784
2,ana_brandine,25039122_545285472508824_4149865287648280576_n...,138.08335841,58.6613147093,151.0,128.843975694,73.2431179783,137.0,123.966246142,74.1573139569,111.0,5.2869212963
3,ana_brandine,25039492_1714465358575359_8547249987241639936_...,192.861972737,16.6466065826,195.0,186.663268176,18.8153630685,188.0,177.462984396,20.1187049951,179.0,0.377996399177
4,ana_brandine,26065304_157340571718200_1337555710505910272_n...,45.3938666706,37.9775345151,37.0,141.085125236,24.2110559957,136.0,114.030282668,33.0500515052,103.0,6.61865695888


In [20]:
#Define Cluster Count
cluster_count = 7

#Define Dataset
data = df.copy(deep=True)
del data["User_Handle"]
del data["URL"]

#Implement K-Means Algortihm
model_kmeans = KMeans(n_clusters=cluster_count, random_state=9001)

#Fit Model and Predict
model_kmeans.fit(data)
y_pred = model_kmeans.predict(data)
df['Prediction'] = model_kmeans.labels_
display(df.head())

,User_Handle,URL,R_Mean,R_STD,R_MED,G_Mean,G_STD,G_MED,B_Mean,B_STD,B_MED,Canny,Prediction
0,ana_brandine,25037786_149039252556440_4293334594247196672_n...,132.561698345,57.6752993548,129.0,114.603278557,56.8672587811,101.0,104.813399871,55.5512984082,85.0,7.39290025088,3
1,ana_brandine,25038631_157219874933240_4514763492913315840_n...,23.5705837674,54.3037901125,5.0,101.522250705,53.6189443253,101.0,170.55649821,51.737780211,181.0,22.7942911784,5
2,ana_brandine,25039122_545285472508824_4149865287648280576_n...,138.08335841,58.6613147093,151.0,128.843975694,73.2431179783,137.0,123.966246142,74.1573139569,111.0,5.2869212963,2
3,ana_brandine,25039492_1714465358575359_8547249987241639936_...,192.861972737,16.6466065826,195.0,186.663268176,18.8153630685,188.0,177.462984396,20.1187049951,179.0,0.377996399177,6
4,ana_brandine,26065304_157340571718200_1337555710505910272_n...,45.3938666706,37.9775345151,37.0,141.085125236,24.2110559957,136.0,114.030282668,33.0500515052,103.0,6.61865695888,5


In [21]:
#Helper function to create folders for Image Clustering
def save_clusters(df, user_imgs, dest_path, label):
    dest_path += label + "/"
    for i in user_imgs.items():
        temp_row = df[df["URL"]==i[0][1]]
        name, pred_folder = i[0][1], str(temp_row['Prediction'].values[0])
        temp_path = dest_path + "Cluster" + pred_folder + "/"
        if not os.path.exists(temp_path):
            os.makedirs(temp_path)
        io.imsave(temp_path+name, i[1])

save_clusters(df, user_imgs, dest_path, "Random_Users")
print("All Images Clustered.")

C:\Users\karan\Documents\Softwares\lib\site-packages\skimage\io\_io.py:132: UserWarning: C:/Users/karan/Desktop/Dropbox/Harvard/Capstone_EmpSirenuse/Clustering_Random_Users/Cluster0/26266872_1754147014894995_2256120881497505792_n.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)


All Images Clustered.


In [40]:
#Percentage of Cluster Presence
cluster_presence = []
for i in list(df['User_Handle'].unique()):
    user_dict = {}
    temp_df = df[df['User_Handle']==i]
    post_count  =  len(temp_df)
    user_dict['User'] = i
    for j in sorted(list(temp_df['Prediction'].unique())):
        user_dict["Cluster_"+str(j)] = len(temp_df[temp_df['Prediction']==j])*100/post_count
    cluster_presence.append(user_dict)

#Create Cluster Presence Dataframe
df_presence = pd.DataFrame(cluster_presence)
df_presence = df_presence.fillna(0)
display(df_presence)

,Cluster_0,Cluster_1,Cluster_2,Cluster_3,Cluster_4,Cluster_5,Cluster_6,User
0,18.888889,15.925926,18.703704,22.777778,5.925926,3.888889,13.888889,ana_brandine
1,0.000000,0.000000,0.000000,100.000000,0.000000,0.000000,0.000000,vicky_regouli
2,3.439803,26.044226,12.285012,18.796069,18.550369,11.179361,9.705160,luismiguelpss
3,0.000000,0.000000,0.000000,100.000000,0.000000,0.000000,0.000000,ilariabiagini
4,0.000000,0.000000,0.000000,100.000000,0.000000,0.000000,0.000000,emnegg
5,3.430079,12.928760,33.245383,28.496042,5.013193,4.749340,12.137203,kerendhahn
6,0.000000,23.076923,30.769231,7.692308,0.000000,15.384615,23.076923,agiorgina
7,100.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,roulamatta
8,0.000000,0.000000,33.333333,0.000000,0.000000,66.666667,0.000000,eremiaheidr
9,100.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,eunhuiheo


In [41]:
#Define Cluster Count
cluster_count = 4

#Define Dataset
data_presence = df_presence.copy(deep=True)
del data_presence["User"]

#Implement K-Means Algortihm
model_kmeans = KMeans(n_clusters=cluster_count, random_state=9001)

#Fit Model and Predict
model_kmeans.fit(data_presence)
df_presence['Prediction'] = model_kmeans.labels_
display(df_presence)

,Cluster_0,Cluster_1,Cluster_2,Cluster_3,Cluster_4,Cluster_5,Cluster_6,User,Prediction
0,18.888889,15.925926,18.703704,22.777778,5.925926,3.888889,13.888889,ana_brandine,0
1,0.000000,0.000000,0.000000,100.000000,0.000000,0.000000,0.000000,vicky_regouli,3
2,3.439803,26.044226,12.285012,18.796069,18.550369,11.179361,9.705160,luismiguelpss,0
3,0.000000,0.000000,0.000000,100.000000,0.000000,0.000000,0.000000,ilariabiagini,3
4,0.000000,0.000000,0.000000,100.000000,0.000000,0.000000,0.000000,emnegg,3
5,3.430079,12.928760,33.245383,28.496042,5.013193,4.749340,12.137203,kerendhahn,0
6,0.000000,23.076923,30.769231,7.692308,0.000000,15.384615,23.076923,agiorgina,0
7,100.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,roulamatta,2
8,0.000000,0.000000,33.333333,0.000000,0.000000,66.666667,0.000000,eremiaheidr,0
9,100.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,eunhuiheo,2
